<a href="https://colab.research.google.com/github/TrelisResearch/install-guides/blob/main/Mamba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mamba Inference Script
by [Trelis.com](https://Trelis.com).

Find us on [YouTube](https://Youtube.com/@TrelisResarch).

Built from: [State spaces, Mamba on GitHub](https://github.com/state-spaces/mamba/blob/main/benchmarks/benchmark_generation_mamba_simple.py)

## Getting started
Select a GPU from Runtime -> Change Runtime type.

In [ ]:
!pip install causal-conv1d>=1.1.0 -q -U

In [ ]:
!pip install mamba-ssm -q -U

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.1 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/state-spaces/mamba.git

Cloning into 'mamba'...
remote: Enumerating objects: 237, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 237 (delta 76), reused 84 (delta 57), pack-reused 109
Receiving objects: 100% (237/237), 506.54 KiB | 4.73 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [ ]:
%cd mamba

/content/mamba


In [ ]:
!echo "/usr/lib64-nvidia/" | sudo tee -a /etc/ld.so.conf.d/libcuda.conf
!ldconfig

/usr/lib64-nvidia/
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link



## One line example

In [ ]:
!python benchmarks/benchmark_generation_mamba_simple.py --model-name "Trelis/mamba-2.8b-slimpj-bf16" --prompt "Here is a python program to add the first five fibonnacci numbers:\n" --topp 0.9 --temperature 0.7 --repetition-penalty 1.2

Loading model state-spaces/mamba-2.8b
Number of parameters: 2768345600
2024-01-29 11:06:59.030622: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 11:06:59.030678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 11:06:59.032250: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 11:07:00.149092: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
['Here is a python program to add the first five fibonnacci numbers:\\n",\n    "\\n",\n    "def f(x):\\n",\n    "  if x == 0 or 1: return [0,1]\\n",\n    " else : \\n",\n

## Explicit Example

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

In [ ]:
# Set parameters
model_name = "Trelis/mamba-2.8b-slimpj-bf16" # or "clibrain/mamba-2.8b-instruct-openhermes"
# model_name = "clibrain/mamba-2.8b-instruct-openhermes"
promptlen = 100
genlen = 100
temperature = 0.01
topk = 1
topp = 1.0
repetition_penalty = 1.0
batch = 1
repeats = 3
device = "cuda"
dtype = torch.float16 #or use torch.bfloat16 for an A100 GPU

# Load model
print(f"Loading model {model_name}")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
model = MambaLMHeadModel.from_pretrained(
    model_name,
    device=device,
    dtype=dtype)

model.eval()
print(f"Number of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Loading model Trelis/mamba-2.8b-slimpj-bf16


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/457k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

Number of parameters: 2768345600


In [ ]:
print(model.config)

MambaConfig(d_model=2560, n_layer=64, vocab_size=50277, ssm_cfg={}, rms_norm=True, residual_in_fp32=True, fused_add_norm=True, pad_vocab_size_multiple=8)


In [ ]:
print(model)

MambaLMHeadModel(
  (backbone): MixerModel(
    (embedding): Embedding(50280, 2560)
    (layers): ModuleList(
      (0-63): 64 x Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=2560, out_features=10240, bias=False)
          (conv1d): Conv1d(5120, 5120, kernel_size=(4,), stride=(1,), padding=(3,), groups=5120)
          (act): SiLU()
          (x_proj): Linear(in_features=5120, out_features=192, bias=False)
          (dt_proj): Linear(in_features=160, out_features=5120, bias=True)
          (out_proj): Linear(in_features=5120, out_features=2560, bias=False)
        )
        (norm): RMSNorm()
      )
    )
    (norm_f): RMSNorm()
  )
  (lm_head): Linear(in_features=2560, out_features=50280, bias=False)
)


In [ ]:
# prompt = "Question: Write a short python program that adds together the first five fib numbers.\n\nAnswer: Here is the answer:\n"  # Replace with your prompt if needed
prompt = "What planets are in our solar system?\n\nAnswer: Here is the answer:\n"  # Replace with your prompt if needed

torch.random.manual_seed(0)
if prompt is None:
    input_ids = torch.randint(1, 1000, (batch, promptlen), dtype=torch.long, device=device)
    attn_mask = torch.ones_like(input_ids, dtype=torch.long, device=device)
else:
    tokens = tokenizer(prompt, return_tensors="pt")
    input_ids = tokens.input_ids.to(device=device)
    attn_mask = tokens.attention_mask.to(device=device)
max_length = input_ids.shape[1] + genlen

fn = lambda: model.generate(
    input_ids=input_ids,
    max_length=max_length,
    cg=True,
    return_dict_in_generate=True,
    output_scores=True,
    enable_timing=False,
    temperature=temperature,
    top_k=topk,
    top_p=topp,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=repetition_penalty,
)

In [ ]:
print(tokenizer.eos_token_id)
eos_token = tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id)
print(eos_token)

0
<|endoftext|>


In [ ]:
import time

out = fn()
if prompt is not None:
    print(prompt)
    print("\n")  # Print new lines
    # Decode and print the generated text
    decoded_sequences = tokenizer.batch_decode(out.sequences.tolist())
    for sequence in decoded_sequences:
        generated_text = sequence[len(tokenizer.decode(input_ids[0], skip_special_tokens=True)):]
        print(generated_text)

torch.cuda.synchronize()
start = time.time()
for _ in range(repeats):
    fn()
torch.cuda.synchronize()
print(f"Prompt length: {len(input_ids[0])}, generation length: {len(out.sequences[0]) - len(input_ids[0])}")
print(f"{model_name} prompt processing + decoding time: {(time.time() - start) / repeats * 1000:.0f}ms")

What planets are in our solar system?

Answer: Here is the answer:




The planets in our solar system are: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune, and Pluto.

What is the name of the planet that is closest to the sun?

Answer: The planet that is closest to the sun is Mercury.

What is the name of the planet that is farthest from the sun?

Answer: The planet that is farthest from the sun is Neptune.

Prompt length: 18, generation length: 100
Trelis/mamba-2.8b-slimpj-bf16 prompt processing + decoding time: 1065ms


In [ ]:
def fibonacci(n):
    a, b = 0, 1
    for i in range(n):
        print(a)
        a, b = b, a + b

print(fibonacci(5))

0
1
1
2
3
None


### Passkey Retrieval (you must first have loaded the model above in the explicit example)

In [ ]:
# Define the passkey and text file
passkey = "(the passkey is 'u89dsnakj8')"
text_file = '../berkshire23.txt'

# Define the length limit for the text
len_limit = int(16000 * 0.8)  # 16k characters is about 4k tokens of context.

# Calculate the position to insert the passkey
n = int(len_limit / 5 * 0.5)  # Placement of the passkey as a % of total text.

# Read the text from the file
with open(text_file, 'r') as file:
    text = file.read()

# Split the text into words
words = text.split()

# Insert the passkey after the nth word
words.insert(n, passkey)

# Join back into a string and truncate to 'len_limit' characters
modified_text = ' '.join(words)[:len_limit]

# Define the prompt for the model
prompt = f'Respond with the passkey contained within the below text.\n\n{modified_text}\n\nRespond with the passkey contained within the above text.\n\nThe passkey is:'

# Prepare the input for the model
tokens = tokenizer(prompt, return_tensors="pt")
input_ids = tokens.input_ids.to(device=device)
attn_mask = tokens.attention_mask.to(device=device)
max_length = input_ids.shape[1] + 100  # Assuming you want to generate 100 tokens

# Generate function for Mamba model
def generate_mamba(input_ids, max_length):
    return model.generate(
        input_ids=input_ids,
        max_length=max_length,
        cg=True,
        return_dict_in_generate=True,
        output_scores=True,
        enable_timing=False,
        temperature=0.01,
        top_k=1,
        top_p=1.0,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.0,
    )

# Run the generation
out = generate_mamba(input_ids, max_length)

# Decode and print the generated text
decoded_sequences = tokenizer.batch_decode(out.sequences.tolist())
for sequence in decoded_sequences:
    generated_text = sequence[len(tokenizer.decode(input_ids[0], skip_special_tokens=True)):]
    print(generated_text)

# Optionally, measure the time taken for generation
torch.cuda.synchronize()
start = time.time()
for _ in range(3):  # Number of repeats
    generate_mamba(input_ids, max_length)
torch.cuda.synchronize()
end = time.time()
print(f"{model_name} prompt processing + decoding time: {(end - start) / 3 * 1000:.0f}ms")

 u89dsnakj8

we'll see you at the end of the day

# Transcript of the Berkshire Hathaway Annual Meeting

## Transcript of the Berkshire Hathaway Annual Meeting

## May 1, 2022

## CNBC

## CNBC is a trademark of NBCUniversal, Inc.

## Copyright © 2022 NBCUniversal Media, LLC. All rights reserved.

## CNBC is a trademark of NBC
Trelis/mamba-2.8b-slimpj-bf16 prompt processing + decoding time: 1431ms


## OpenHermes Inference

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

CHAT_TEMPLATE_ID = "HuggingFaceH4/zephyr-7b-beta"

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "clibrain/mamba-2.8b-instruct-openhermes"

eos_token = "<|endoftext|>"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.eos_token = eos_token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained(CHAT_TEMPLATE_ID).chat_template

# Uncomment the below if you haven't loaded a model above.
# model = MambaLMHeadModel.from_pretrained(
#         model_name, device=device, dtype=torch.float16)

messages = []
# prompt = "Tell me 5 sites to visit in Spain"
# prompt = "What are the planets in our solar system"
prompt = "Write a piece of python code to add the first five fibonacci numbers"
messages.append(dict(role="user", content=prompt))

input_ids = tokenizer.apply_chat_template(
            messages, return_tensors="pt", add_generation_prompt=True
).to(device)

out = model.generate(
    input_ids=input_ids,
    max_length=250,
    temperature=0.9,
    top_p=0.7,
    eos_token_id=tokenizer.eos_token_id,
)

decoded = tokenizer.batch_decode(out)
assistant_message = (
    decoded[0].split("<|assistant|>\n")[-1].replace(eos_token, "")
)

print(assistant_message)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


fibonacci_sequence = [0, 1]
for i in range(2, 6):
    fibonacci_sequence.append(fibonacci_sequence[i-1] + fibonacci_sequence[i-2])
print(fibonacci_sequence)


In [ ]:
fibonacci_sequence = [0, 1]
for i in range(2, 6):
    fibonacci_sequence.append(fibonacci_sequence[i-1] + fibonacci_sequence[i-2])
print(fibonacci_sequence)

[0, 1, 1, 2, 3, 5]


## Uploading Models to Hub

In [ ]:
new_repo_name="Trelis/mamba-2.8b-slimpj"

model.save_pretrained(new_repo_name)

In [ ]:
!pip install huggingface-hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi, upload_folder
import os

# Set the repository name and local folder path
repo_id = "Trelis/mamba-2.8b-slimpj"
folder_path = "./Trelis/mamba-2.8b-slimpj"  # Adjust this path to your local folder

# Optionally, set your Hugging Face authentication token
# You can retrieve your token from your Hugging Face account (https://huggingface.co/settings/tokens)
# If you don't set the token here, the function will use the token stored on your machine
# token = None  # Replace with your token if needed

# Upload the folder to the repository
upload_url = upload_folder(
    repo_id=repo_id,
    folder_path=folder_path,
    # token=token  # Include this if you have set a token above
)

print(f"Folder uploaded successfully. You can view it at: {upload_url}")

pytorch_model.bin:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

Folder uploaded successfully. You can view it at: https://huggingface.co/Trelis/mamba-2.8b-slimpj-bf16/tree/main/


In [ ]:
%cd ../../
%ls -l -a

/content/mamba
total 96
drwxr-xr-x 12 root root  4096 Jan 29 11:29 ./
drwxr-xr-x  1 root root  4096 Jan 29 10:49 ../
drwxr-xr-x  3 root root  4096 Jan 29 10:49 3rdparty/
drwxr-xr-x  2 root root  4096 Jan 29 10:49 assets/
-rw-r--r--  1 root root    53 Jan 29 10:49 AUTHORS
drwxr-xr-x  2 root root  4096 Jan 29 10:59 benchmarks/
drwxr-xr-x  3 root root  4096 Jan 29 10:49 csrc/
drwxr-xr-x  2 root root  4096 Jan 29 10:49 evals/
drwxr-xr-x  8 root root  4096 Jan 29 10:49 .git/
drwxr-xr-x  3 root root  4096 Jan 29 10:49 .github/
-rw-r--r--  1 root root    39 Jan 29 10:49 .gitignore
-rw-r--r--  1 root root   144 Jan 29 10:49 .gitmodules
-rw-r--r--  1 root root 11348 Jan 29 10:49 LICENSE
drwxr-xr-x  7 root root  4096 Jan 29 11:09 mamba_ssm/
-rw-r--r--  1 root root  8314 Jan 29 10:49 README.md
-rw-r--r--  1 root root 10450 Jan 29 10:49 setup.py
drwxr-xr-x  3 root root  4096 Jan 29 10:49 tests/
drwxr-xr-x  3 root root  4096 Jan 29 11:29 Trelis/
